In [16]:
import pandas as pd
import os

#import csvs, join many to one on "Location" and "CCode" variables

cowdata = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/COW-country-codes-edited.csv")
esricountries = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/World_Countries.csv")
print(esricountries.head())

print(esricountries.shape) # (651, 9)
print(cowdata.shape) # 243 x 4

#rename cols in cowdata 
cowdata = cowdata.rename(columns={"StateAbb": "LocationState", "CCode": "LocationCode"})
print(cowdata.head())

   FID      COUNTRY ISO_CC CONTINENT          LAND_TYPE  LAND_RANK  \
0    1  Afghanistan    AFG      Asia       Primary land          5   
1    2      Albania    ALB    Europe       Primary land          5   
2    3      Albania    ALB    Europe  Very small island          1   
3    4      Albania    ALB    Europe       Small island          2   
4    5      Algeria    DZA    Africa       Small island          2   

    COUNTRYAFF  SHAPE_Length  SHAPE_Area  
0  Afghanistan     59.141644   62.580697  
1      Albania     12.826590    3.083112  
2      Albania      0.080608    0.000058  
3      Albania      0.175080    0.000764  
4      Algeria      0.044204    0.000060  
(651, 9)
(243, 4)
  LocationState  LocationCode     StateNme Note
0           AFG           700  Afghanistan  NaN
1           ALB           339      Albania  NaN
2           ALB           339      Albania  NaN
3           ALG           615      Algeria  NaN
4           AND           232      Andorra  NaN


In [17]:
#many-to-one merge in pandas, left join
#cowcountriesmerged = pd.merge(cowdata, esricountries, how="left", left_on="StateNme", right_on="COUNTRY")
cowcountriesmerged = cowdata.merge(esricountries, how='left', left_on="StateNme", right_on="COUNTRY", indicator=True)
print(cowcountriesmerged.shape)
print(cowcountriesmerged.head(10))
#column_names = list(joinedmiptocow.columns)
#print(column_names)

(612, 14)
  LocationState  LocationCode     StateNme Note  FID      COUNTRY ISO_CC  \
0           AFG           700  Afghanistan  NaN  1.0  Afghanistan    AFG   
1           ALB           339      Albania  NaN  2.0      Albania    ALB   
2           ALB           339      Albania  NaN  3.0      Albania    ALB   
3           ALB           339      Albania  NaN  4.0      Albania    ALB   
4           ALB           339      Albania  NaN  2.0      Albania    ALB   
5           ALB           339      Albania  NaN  3.0      Albania    ALB   
6           ALB           339      Albania  NaN  4.0      Albania    ALB   
7           ALG           615      Algeria  NaN  5.0      Algeria    DZA   
8           ALG           615      Algeria  NaN  6.0      Algeria    DZA   
9           ALG           615      Algeria  NaN  7.0      Algeria    DZA   

  CONTINENT          LAND_TYPE  LAND_RANK   COUNTRYAFF  SHAPE_Length  \
0      Asia       Primary land        5.0  Afghanistan     59.141644   
1    Euro

In [18]:
#to csv - this is a file that has all of state country codes (with nulls) and country names
cowcountriesmerged.to_csv('joined-countriestocow.csv')
# the file above still needs to merge to MIP data
# possibly x,y locations from dispnum variable matching to COW datasets


In [19]:
#additional join/merge x 2 - cowcountriesmerged to mipdata based on a) geo location and b) political
#redo this for geo data
mipdata = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/EditedMIPData.csv")
print(mipdata.head()) #mipdata edited manually, now will match based on Location Code (code for intervention location, not political necessarily)
print(mipdata['Location_Code'].dtype)
#print(cowcountriesmerged.dtypes)


  State2 State3 State4 State5 State6 State7  State2_Code  State3_Code  \
0    FRN    NaN    NaN    NaN    NaN    NaN        220.0          NaN   
1    LIB    NaN    NaN    NaN    NaN    NaN        620.0          NaN   
2    MEX    NaN    NaN    NaN    NaN    NaN         70.0          NaN   
3    SPN    FRN    NaN    NaN    NaN    NaN        220.0        230.0   
4    SPN    NaN    NaN    NaN    NaN    NaN        230.0          NaN   

   State4_Code  State5_Code  ...  US nGDP capita   US rGDP capita  \
0          NaN          NaN  ...            82.77         1486.37   
1          NaN          NaN  ...            94.12         1611.78   
2          NaN          NaN  ...            96.72         1658.16   
3          NaN          NaN  ...            96.72         1658.16   
4          NaN          NaN  ...            97.73         1668.70   

   US rGDPCChange  UNRegN                     UNRegT UNSUBREG  \
0             NaN   419.0  Latin America & Caribbean     13.0   
1            8.4

In [20]:
# need to change so dtypes match for below join - change both to float
# LocationCode in cowcountriesmerged is int64 and Location_Code / State2_Code are float64
# example format: convert column "a" of a DataFrame >> df['Discount']=df.Discount.replace('',np.nan).astype(float)

mipdata['Location_Code'] = pd.to_numeric(mipdata['Location_Code'], errors='coerce')
print(mipdata['Location_Code'].dtype)

cowcountriesmerged['LocationCode'] = cowcountriesmerged['LocationCode'].astype(float)
print(cowcountriesmerged['LocationCode'].dtype)


float64
float64


In [21]:
# political:
mipcowcountriesmerged_polit = cowcountriesmerged.merge(mipdata, how='left', left_on="LocationCode", right_on="State2_Code")
# geo:
mipcowcountriesmerged_geo = cowcountriesmerged.merge(mipdata, how='left', left_on="LocationCode", right_on="Location_Code")

#to csv - this is a file that has state 2 countries matched from esri
#mipcowcountriesmerged_polit.to_csv('countriesmerged_political.csv')
mipcowcountriesmerged_geo.to_csv('countriesmerged_geo.csv') #check these csvs - at this point have lost data 


In [22]:
mipcowcountriesmerged_polit = mipcowcountriesmerged_polit.rename(columns={'StateNme': 'LocationName', 'COUNTRY': 'EsriCountryName', 'CONTINENT':'EsriContinent'})
mipcowcountriesmerged_geo = mipcowcountriesmerged_geo.rename(columns={'StateNme': 'LocationName', 'COUNTRY': 'EsriCountryName', 'CONTINENT':'EsriContinent'})
print(mipcowcountriesmerged_geo.head())

  LocationState  LocationCode LocationName Note  FID EsriCountryName ISO_CC  \
0           AFG         700.0  Afghanistan  NaN  1.0     Afghanistan    AFG   
1           AFG         700.0  Afghanistan  NaN  1.0     Afghanistan    AFG   
2           AFG         700.0  Afghanistan  NaN  1.0     Afghanistan    AFG   
3           AFG         700.0  Afghanistan  NaN  1.0     Afghanistan    AFG   
4           ALB         339.0      Albania  NaN  2.0         Albania    ALB   

  EsriContinent     LAND_TYPE  LAND_RANK  ... US nGDP capita   US rGDP capita  \
0          Asia  Primary land        5.0  ...        32817.92        43592.71   
1          Asia  Primary land        5.0  ...        37099.83        46496.98   
2          Asia  Primary land        5.0  ...        37099.83        46496.98   
3          Asia  Primary land        5.0  ...        56700.89        54109.73   
4        Europe  Primary land        5.0  ...         2763.28        17066.63   

   US rGDPCChange UNRegN  UNRegT UNSUB

In [26]:
#de dup mipcowcountriesmerged_geo and mipcowcountriesmerged_polit

print(mipcowcountriesmerged_geo[['LocationState','Start_year', 'Description']].head(10))
print(mipcowcountriesmerged_geo.shape)

dedup_geo = mipcowcountriesmerged_geo.drop_duplicates( 
  subset = ['Start_year', 'Description'], 
  keep = 'last').reset_index(drop = True) 
  
# print latest dataframe 
print(dedup_geo[['LocationState','Start_year', 'Description']].head(10))
print(dedup_geo.shape) # 552 lines by 79 columns

dedup_polit = mipcowcountriesmerged_polit.drop_duplicates( 
  subset = ['Start_year', 'Description'], 
  keep = 'last').reset_index(drop = True) 
  
# print latest dataframe 
print(dedup_polit[['LocationState','Start_year', 'Description']].head(10))
print(dedup_polit.shape) # 565 lines by 79 columns

#export as new csv - manually edit any blanks
dedup_geo.to_csv('dedup_editedXYmipdata_geo.csv')


  LocationState  Start_year                                        Description
0           AFG      1998.0  US uses cruise missiles to attack suspected te...
1           AFG      2001.0  Invasion. ISAF in Afghanistan to help stabiliz...
2           AFG      2001.0                         Operation Enduring Freedom
3           AFG      2015.0                       Operation Freedom's Sentinel
4           ALB      1958.0  Albanian fighters force down a US Air Force je...
5           ALB      1997.0  Albania. On March 15, 1997, President Clinton ...
6           ALB      1958.0  Albanian fighters force down a US Air Force je...
7           ALB      1997.0  Albania. On March 15, 1997, President Clinton ...
8           ALB      1958.0  Albanian fighters force down a US Air Force je...
9           ALB      1997.0  Albania. On March 15, 1997, President Clinton ...
(2545, 79)
  LocationState  Start_year                                        Description
0           AFG      1998.0  US uses crui

In [27]:
#new, manually edited
dedup_geo = pd.read_csv('/Users/annafeldman/Desktop/GIS Final/dedup_editedXYmipdata_geo_manualedit.csv')

esricities = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/World_Capitals.csv")
esricities = esricities[["OBJECTID", "CITY_NAME", "GMI_ADMIN", "ADMIN_NAME", "FIPS_CNTRY", "CNTRY_NAME", "STATUS", "POINT_X", "POINT_Y"]]
esricities = esricities.rename(columns={'OBJECTID': 'ESRICity_ID', 'CITY_NAME': 'ESRICity_Name', 
                                        'CNTRY_NAME': 'ESRICity_CountryName', 'POINT_X': 'EsriCity_X', 'POINT_Y': 'EsriCity_Y'})
print(esricities.head())
print(esricities.shape) #(229, 9)

print(dedup_geo.shape) # (565, 79), working with de dupped and manually edited file
#now going to use esricities (edited World Capitals csv) with it


   ESRICity_ID ESRICity_Name GMI_ADMIN                       ADMIN_NAME  \
0         1757     Abu Dhabi   ARE-AZA                         Abu Zaby   
1          880         Abuja   NGA-FCT  Abuja Capital Territory (Abuja)   
2          843         Accra   GHA-GAC                    Greater Accra   
3           23     Adamstown       PCN                     Pitcairn Is.   
4         1384   Addis Ababa   ETH-SHE                            Shewa   

  FIPS_CNTRY  ESRICity_CountryName                           STATUS  \
0         AE  United Arab Emirates  National and provincial capital   
1         NI               Nigeria  National and provincial capital   
2         GH                 Ghana  National and provincial capital   
3         PC          Pitcairn Is.  National and provincial capital   
4         ET              Ethiopia  National and provincial capital   

   EsriCity_X  EsriCity_Y  
0   54.370998   24.476004  
1    7.256385    9.057267  
2   -0.200925    5.558562  
3 -130.103

In [28]:
dedup_geo =dedup_geo.merge(esricities, how='left', left_on="EsriCountryName", right_on="ESRICity_CountryName")
print(dedup_geo.shape) #added columns, now 88 columns by 565 lines
dedup_polit = dedup_polit.merge(esricities, how='left', left_on="EsriCountryName", right_on="ESRICity_CountryName")
print(dedup_polit.shape) #added columns, now 88 columns by 565 lines

dedup_geo.to_csv('citiesXYmipdatamerged_geo.csv')
# dedup_polit.to_csv('citiesXYmipdatamerged_polit.csv') #next, manually cleaned political csv, then geo csv

(565, 89)
(565, 88)


In [31]:
#count occurrences of each country and add to new column - need to repeat below with geo csv
#data['column_name'].value_counts()
#countdata = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/citiesXYmipdatamerged_polit_manualedit.csv")
countdata = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/citiesXYmipdatamerged_geo_manualedit.csv")
#print(countdata.head())

#print(countdata['EsriCountryName'].value_counts())
country_counts = countdata['EsriCountryName'].value_counts()
print(country_counts) #list of countries with their counts

EsriCountryName
China          47
Mexico         30
Cuba           26
Russia         25
North Korea    21
               ..
Niger           1
Netherlands     1
Mozambique      1
Morocco         1
Zimbabwe        1
Name: count, Length: 96, dtype: int64


In [32]:
# - need to repeat below with geo csv
countdata = countdata.merge(country_counts, how="left", left_on="EsriCountryName", right_on="EsriCountryName")
print(countdata.head())
countdata.to_csv('addcount_final_geo.csv')

   Index  Unnamed: 0 LocationState  LocationCode LocationName Note  FID  \
0      0           0           AFG           700  Afghanistan  NaN  1.0   
1      1           1           AFG           700  Afghanistan  NaN  1.0   
2      2           2           AFG           700  Afghanistan  NaN  1.0   
3      3           3           AFG           700  Afghanistan  NaN  1.0   
4      4           4           ALB           339      Albania  NaN  4.0   

  EsriCountryName ISO_CC EsriContinent  ... ESRICity_ID  ESRICity_Name  \
0     Afghanistan    AFG          Asia  ...      2063.0          Kabul   
1     Afghanistan    AFG          Asia  ...      2063.0          Kabul   
2     Afghanistan    AFG          Asia  ...      2063.0          Kabul   
3     Afghanistan    AFG          Asia  ...      2063.0          Kabul   
4         Albania    ALB        Europe  ...      1129.0         Tirana   

  GMI_ADMIN  ADMIN_NAME  FIPS_CNTRY ESRICity_CountryName  \
0   AFG-KAB       Kabol          AF         

In [ ]:
# #delete duplicates after manual editing
# import pandas as pd 
# #reimport csv
# #delete entries where year and description are identical - true dups 'Start_year', 'Description'
# dedup = pd.read_csv("/Users/annafeldman/Desktop/GIS Final/citiesXYmipdatamerged_geo_citiesedited.csv")
# print(dedup[['LocationState','Start_year', 'Description']].head(10))

# newdf = dedup.drop_duplicates( 
#   subset = ['Start_year', 'Description'], 
#   keep = 'last').reset_index(drop = True) 
  
# # print latest dataframe 
# print(newdf[['LocationState','Start_year', 'Description']].head(10))

# #export as new csv
# newdf.to_csv('dedup_editedXYmipdata.csv')


  LocationState  Start_year                                        Description
0           USA        1798  Undeclared Naval War with France. This contest...
1           LIB        1801  Tripoli. The First Barbary War included the US...
2           MEX        1806  Captain Z. M. Pike, with a platoon of troops, ...
3           USA        1806  Gulf of Mexico. American gunboats operated fro...
4           USA        1810  West Florida (Spanish territory). Governor Wil...
5           USA        1812  Amelia Island and other parts of east Florida,...
6           USA        1812  War of 1812. On June 18, 1812, the United Stat...
7           SPN        1813  West Florida (Spanish territory). On authority...
8           SPN        1813  West Florida (Spanish territory). On authority...
9           SPN        1813  West Florida (Spanish territory). On authority...
  LocationState  Start_year                                        Description
0           USA        1798  Undeclared Naval War wi